In [ ]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import sys
import yfinance as yf
tqdm.pandas()
sys.path.insert(2,'..')


In [3]:
import sys
sys.path.insert(2,'..')


In [4]:
import functions
import selection
import DataPull
import annualized_rv as arv

In [5]:
dataPulled = DataPull.pull_microsoft_option_data()


In [6]:
selected_data= selection.filter_options(dataPulled)
selected_data['implied_volatility'] = selected_data.progress_apply(functions.implied_volatility,axis =1 )


  0%|▍                                                                            | 301/60723 [00:01<03:24, 296.09it/s]X:\github\ThesisProject\Jupyterlab\..\functions.py:75: RuntimeWarning: overflow encountered in scalar power
  d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
X:\github\ThesisProject\Jupyterlab\..\functions.py:8: RuntimeWarning: overflow encountered in scalar power
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
  5%|███▌                                                                        | 2857/60723 [00:13<03:02, 317.63it/s]C:\Users\Ahmed\miniconda3\envs\myenv\Lib\site-packages\scipy\optimize\_zeros_py.py:306: RuntimeWarning: overflow encountered in scalar divide
  newton_step = fval / fder
X:\github\ThesisProject\Jupyterlab\..\functions.py:75: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
X:\github\ThesisProject\Jupyterlab\..\

KeyboardInterrupt: 

In [ ]:
dailyAverageIV= functions.average_daily_implied_volatility(selected_data)


In [ ]:
stock_data = yf.download('MSFT', start='2016-01-01', end='2023-12-31')
stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()

window = 21
stock_data['1M Realized Volatility'] = arv.annualized_volatility(stock_data['Daily Return'],window)


#plotting

plt.figure(figsize=(12, 5))
plt.plot(dailyAverageIV['date'], dailyAverageIV['average_iv'],label='Average Implied Volatility')
plt.plot(stock_data.index, stock_data['1M Realized Volatility'], label='1M Realized Volatility')

plt.title('Average Implied Volatility Over Time')
plt.xlabel('Date')
plt.ylabel('Average Implied Volatility')
plt.grid(True)
plt.legend()

#compare with actual volatiltiy

plt.show()

In [ ]:
# current_dir = os.getcwd()
# data_dir = os.path.join(current_dir, 'data')
# os.makedirs(data_dir, exist_ok=True)
# series.to_pickle(os.path.join(data_dir, 'average_iv_series_with_date_microsoft.pkl'))


In [ ]:
dailyAverageIV

In [ ]:
car = 'm1sft_mean_iv_2017_220222'
dailyAverageIV.to_pickle(os.path.join('historicImpliedVolData', car+'.pkl',))

In [ ]:
# in function form for usability 

def dailymeanVolpkl(data : pd.DataFrame ,name : str) -> None:
    selected_data= selection.filter_options(data) # filters data based on time to maturity/moneyness 
    selected_data['implied_volatility'] = selected_data.progress_apply(functions.implied_volatility,axis =1 ) #calcs daily iv
    
    dailyAverageIV= functions.average_daily_implied_volatility(selected_data) #means the IV
    dailyAverageIV.to_pickle(os.path.join('historicImpliedVolData', name+'.pkl'))
    print(name , "done")

In [ ]:
dailymeanVolpkl(DataPull.pull_alphabet_option_data(),'goog_mean_iv_2017_2022')

In [ ]:
dailymeanVolpkl(DataPull.pull_apple_option_data(),'aapl_mean_iv_2017_2022')

In [ ]:
dailymeanVolpkl(DataPull.pull_nasdaq100_option_data(),'ndx_mean_iv_2017_2022')

In [ ]:
dailymeanVolpkl(DataPull.pull_sp500_option_data(),'spc_mean_iv_2017_2022')